In [1]:
import time
import torch
from torch import nn
#from torch.optim import Adam
from torch.optim import SGD
from torchtext.data import Field, BucketIterator
import copy
from torch.nn.utils.rnn import pack_padded_sequence

In [2]:
from torchtext.datasets import SequenceTaggingDataset

In [3]:
import csv

In [4]:
import pandas as pd

import numpy as np 

data_file = pd.read_csv('train', quoting=csv.QUOTE_NONE,sep=" ", names= ['index', 'word', 'tag'])

In [5]:
# convert 
data_array = data_file.to_numpy()

In [6]:
### store the lists of sentences into a big list 
 
def data_to_stream(data):
    data_stream = []
    i = 0 
    for index, word, tag in data:
        if index == 1:
            temp = []
            temp.append([index,word, tag])
            
        else:
            temp.append([index ,word,tag ])
            
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            data_stream.append(temp)
        
        i += 1
            
    return data_stream 

In [7]:
data_stream = data_to_stream(data_array)

In [29]:
def x_y(indexical_train):
    x = []
    y = []
    for sentence in indexical_train:
        temp_x = []
        temp_y = []
        for word in sentence:
            temp_x.append(word[1])
            temp_y.append(word[2])
        x.append(temp_x)
        y.append(temp_y)
    return x,y
        

In [30]:
train_word,train_tag = x_y(data_stream)

In [10]:
all_data_list = list(zip(train_word, train_tag ))

In [11]:
word_set = set([str(word) for seq in train_word for word in seq])

word_vocab = ['<pad>']+sorted(word_set)+['<unk>']

word_to_index = {word:idx for idx,word in enumerate(word_vocab)}

index_to_word = {idx:word for idx,word in enumerate(word_vocab)}

In [12]:
tag_set = set([str(tag)for sentence in train_tag for tag in sentence])
tag_vocab =  sorted(tag_set)+['<pad>'] 
tag_to_index = {tag:idx for idx,tag in enumerate(tag_vocab)}
index_to_tag = {idx:tag for idx,tag in enumerate(tag_vocab)}

In [13]:
def indexical_sentence_word(sentence,word_to_index):
    w=[]
    for word in sentence:
        if word in word_to_index:
            w.append(word_to_index[word])
        else:
            w.append(word_to_index['<unk>'])
    #w_tensor = torch.Tensor(w)
    return w
        
    

In [14]:
def indexical_sentence_tag(sentence,tag_to_index):
    t = []
    for tag in sentence:
        t.append(tag_to_index[tag])
    #t_tensor = torch.Tensor(t)
    return t

In [83]:
def indexical_x_y(all_data_list):
    x_train = []
    y_train = []
    for item in all_data_list:
        words,tags = item[0],item[1]
        seq_tensor = indexical_sentence_word(words,word_to_index)
        tags_tensor = indexical_sentence_tag(tags,tag_to_index)
        x_train.append(seq_tensor)
        y_train.append(tags_tensor)
    return x_train,y_train

In [19]:
tag_to_index

{'B-LOC': 0,
 'B-MISC': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-MISC': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8,
 '<pad>': 9}

In [84]:
x_train, y_train = indexical_x_y(all_data_list)

In [40]:
y_train

[[2, 8, 1, 8, 8, 8, 1, 8, 8],
 [3, 7],
 [0, 8],
 [8,
  2,
  6,
  8,
  8,
  8,
  8,
  8,
  8,
  1,
  8,
  8,
  8,
  8,
  8,
  1,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8],
 [0,
  8,
  8,
  8,
  8,
  2,
  6,
  8,
  8,
  8,
  3,
  7,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  0,
  8,
  8,
  8,
  8,
  8,
  8,
  8],
 [8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  2,
  8,
  8,
  8,
  3,
  7,
  7,
  7,
  8,
  8,
  8,
  8,
  8],
 [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 6, 8],
 [8,
  8,
  8,
  8,
  8,
  8,
  8,
  2,
  8,
  8,
  3,
  7,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8],
 [3,
  8,
  1,
  8,
  8,
  8,
  8,
  0,
  8,
  0,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  1,
  5,
  5,
  8,
  1,
  8,
  8,
  8,
  8,
  8,
  8],
 [8,
  3,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  2,
  8,
  8,

In [17]:
sentence_lenth = [len(sen) for sen in x_train]

longest = max(sentence_lenth)

In [113]:
word_pad_index = word_to_index['<pad>']
tag_pad_index = tag_to_index['<pad>']

In [114]:
word_pad_index

0

In [20]:
tag_pad_index

9

In [21]:
len(word_to_index)

23626

In [22]:
sentence_lenth = [len(sen) for sen in x_train]
longest = max(sentence_lenth)

In [23]:
def padding_x_y(x_train,y_train):
    #<pad> is 0
    sentence_lenth = [len(sen) for sen in x_train]
    size = len(x_train)
    padded_x = np.zeros((int(size),int(longest)))
    padded_y = np.zeros((int(size),int(longest)))
    for i,x_len in enumerate(sentence_lenth):
        seq = x_train[i]
        padded_x[i,0:x_len] = seq
    for i,y_len in enumerate(sentence_lenth):
        seq_y = y_train[i]
        padded_y[i,0:y_len] = seq_y
    
    return padded_x,padded_y

In [24]:
embedding = nn.Embedding(len(word_to_index), 100, padding_idx = word_pad_index)

In [25]:
x_pad,y_pad = padding_x_y(x_train,y_train)
x_padded = torch.from_numpy(x_pad)
y_padded = torch.from_numpy(y_pad)

x_padded = x_padded.type(torch.LongTensor) 
y_padded = y_padded.type(torch.LongTensor)

In [29]:
tag_to_index

{'B-LOC': 0,
 'B-MISC': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-MISC': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8,
 '<pad>': 9}

In [30]:
len(x_pad)

14987

In [31]:
len(embedding(torch.LongTensor(x_train[0])))

9

In [32]:
dev_file = pd.read_csv('dev', quoting=csv.QUOTE_NONE, sep=" ", names= ['index', 'word', 'tag'])

In [33]:
#dev_file
# convert 
dev_array = dev_file.to_numpy()
dev_stream = data_to_stream(dev_array)


In [54]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence


In [62]:
def list_to_array(list_data):
    data = copy.deepcopy(list_data)
    for i in range(len(data)):
        data[i] = torch.from_numpy(np.array(data[i]))
    return data

In [102]:
x_train = list_to_array(x_train)

In [103]:
y_train = list_to_array(y_train)

In [107]:
from torch.utils.data import Dataset, DataLoader
## refer to pytorch tutorials on how to inherit from Dataset class


def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=0)

    return xx_pad, yy_pad, x_lens, y_lens

dataset = list(zip(x_train, y_train))
data_loader = DataLoader(dataset=dataset, batch_size=2, shuffle=True, collate_fn=pad_collate)


In [109]:
for xx_pad, yy_pad, x_lens, y_lens in data_loader:
    print(xx_pad, yy_pad, x_lens, y_lens)
    break

tensor([[ 7659, 23339, 22010, 19580, 22570, 20992, 13888, 13691, 21462, 19434,
         21140, 14233, 18624, 16868, 19957,   125,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [11995, 17927,  4310,  6448,  8298, 21152, 23096, 22574, 21642, 21842,
           557, 19008, 14454, 22570, 21293, 19340,    14,   393,  7098,    15,
         16525, 19434, 22570, 22110,   125]]) tensor([[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [8, 8, 0, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8,
         8]]) [16, 25] [16, 25]


In [110]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.elu = nn.ELU()
        
    def forward(self, x_padded, y_padded, x_lens, y_lens):
        #pass text through embedding layer
        x_embed = self.dropout(self.embedding(x_padded))
        
        x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=True, enforce_sorted=False)
        #pass embeddings into LSTM
        output_packed, (hidden, cell) = self.lstm(x_packed)
        output_padded, output_lengths = pad_packed_sequence(output_packed, batch_first=True)
        #we use our outputs to make a prediction of what the tag should be
        outputs = self.elu(output_padded)
        outputs = self.fc(self.dropout(outputs))
    
        
        return outputs

In [115]:
INPUT_DIM = len(word_to_index)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = word_pad_index

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [116]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

In [117]:
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [118]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,161,448 trainable parameters


In [119]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [120]:
#print(model.embedding.weight.data)

In [121]:
### define optimizer fuction 
optimizer = SGD(model.parameters(), lr=0.05)

In [122]:
### define loss function 
TAG_PAD_IDX = tag_to_index['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [142]:
### train the model 
def train(model, data_loader, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for (x_padded, y_padded, x_lens, y_lens) in data_loader:
        
        text = x_padded
        tags = y_padded
        x_lens = x_lens
        y_lens = y_lens
        
        
        optimizer.zero_grad()
        predictions = model(x_padded, y_padded, x_lens, y_lens)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(data_loader),epoch_acc / len(data_loader)

In [143]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.word
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [144]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [ ]:
N_EPOCHS = 2

best_train_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, data_loader, optimizer, criterion, TAG_PAD_IDX)
    #valid_loss, valid_acc = evaluate(model, corpus.val_iter, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 37s
	Train Loss: 0.624 | Train Acc: 85.98%


In [140]:
for i in data_loader:
    print(len(i))
    break

4


In [141]:
i

(tensor([[13455,  8708,  3284,  1498,  2092,  1498,  2092,  2497,  3653],
         [  114,  7805,  8672, 19285,    14,  3607,    15,  2017,  3219]]),
 tensor([[2, 6, 8, 8, 8, 8, 8, 8, 8],
         [8, 0, 4, 8, 8, 8, 8, 8, 8]]),
 [9, 9],
 [9, 9])

In [132]:
#model.load_state_dict(torch.load('tut1-model.pt'))

#index = 0
dev_sentences = []
dev_actual_tags = []
for i in range(len(corpus.val_dataset)):
    
    sentence_words = vars(corpus.val_dataset[i])['word']
    dev_sentences.append(sentence_words)
    actual_tag = vars(corpus.val_dataset[i])['tag']
    dev_actual_tags.append(actual_tag)

    


NameError: name 'corpus' is not defined

In [36]:
#sentence_tag
#dev_sentences

In [43]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    #tokens = [token for token in sentence]

    tokens = sentence
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    #print(top_predictions)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [45]:
#from seqeval.metrics import classification_report, f1_score

In [46]:
#dev_sentences[1]

In [47]:
whole_pred_tags = []

for s in dev_sentences:
    
    _, pred_tags, _ = tag_sentence(model, device, s, corpus.word_field, corpus.tag_field)
    
    whole_pred_tags.append(pred_tags)
    
    

In [48]:
s_1 = ['london', '1996-08-30']
_,pred_t, _ = tag_sentence(model, device, s_1, corpus.word_field, corpus.tag_field)

In [49]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [50]:
whole_pred_tags_f = flatten(whole_pred_tags)

In [51]:
dev_actual_tags_f = flatten(dev_actual_tags)

In [52]:
#whole_pred_tags_f

In [53]:
from sklearn.metrics import classification_report
print(classification_report(dev_actual_tags_f, whole_pred_tags_f))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00      1837
      B-MISC       0.00      0.00      0.00       922
       B-ORG       0.00      0.00      0.00      1341
       B-PER       0.00      0.00      0.00      1842
       I-LOC       0.00      0.00      0.00       257
      I-MISC       0.00      0.00      0.00       346
       I-ORG       0.00      0.00      0.00       751
       I-PER       0.00      0.00      0.00      1307
           O       0.83      1.00      0.91     42975

    accuracy                           0.83     51578
   macro avg       0.09      0.11      0.10     51578
weighted avg       0.69      0.83      0.76     51578



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
#corpus.tag_field.vocab.itos[.item()

In [56]:
import torchtext.vocab as vocab


custom_embeddings = vocab.Vectors(name = 'glove.6B.100d')

100%|█████████▉| 399999/400000 [00:18<00:00, 21875.81it/s]


In [62]:
import torchtext.vocab as vocab


In [80]:

word_field = Field()
tag_field = Field(unk_token=None)
train_dataset,val_dataset = SequenceTaggingDataset.splits(
path='.',
train="train_out.tsv",
validation="dev_out.tsv",
fields=(("word",word_field), ("tag",tag_field)))
word_field.build_vocab(train_dataset, min_freq=1)
tag_field.build_vocab(train_dataset)
# create iterator for batch input
train_iter, val_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset),
    batch_size=32
)

In [84]:
for i in train_iter:
    print(vars(i))
    break

{'batch_size': 32, 'dataset': <torchtext.datasets.sequence_tagging.SequenceTaggingDataset object at 0x000001D63BA02E80>, 'fields': dict_keys(['word', 'tag']), 'input_fields': ['word', 'tag'], 'target_fields': [], 'word': tensor([[  447,  3606,   542,  ...,  3265,  1483,    86],
        [ 1305,   136,   448,  ...,    18,  1834,  4042],
        [  128,     1,     4,  ..., 16727,   665,    26],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]]), 'tag': tensor([[4, 2, 1,  ..., 1, 4, 1],
        [6, 1, 1,  ..., 1, 6, 1],
        [1, 0, 1,  ..., 3, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}


In [88]:
len(i.tag[0])

32

In [86]:
word_field.build_vocab(train_dataset, vectors = custom_embeddings)
word_pad_idx = word_field.vocab.stoi[word_field.pad_token]
tag_pad_idx = tag_field.vocab.stoi[tag_field.pad_token]

In [89]:
INPUT_DIM = len(word_field.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = corpus.word_pad_idx

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [90]:
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(23626, 100, padding_idx=1)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [99]:
model.embedding.weight.data.copy_(word_field.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0823,  0.3968,  0.6859,  ...,  0.0866, -0.0148, -0.1637],
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [-0.5832, -0.5807,  0.3504,  ...,  0.4043, -0.0192,  0.0945]])

In [100]:
optimizer = SGD(model.parameters(), lr=0.05)

In [102]:
TAG_PAD_IDX = tag_field.vocab.stoi[tag_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [104]:
N_EPOCHS = 8

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 52s
	Train Loss: 0.472 | Train Acc: 84.97%
	 Val. Loss: 1.040 |  Val. Acc: 77.77%
Epoch: 02 | Epoch Time: 2m 18s
	Train Loss: 0.441 | Train Acc: 86.06%
	 Val. Loss: 0.992 |  Val. Acc: 78.34%
Epoch: 03 | Epoch Time: 1m 33s
	Train Loss: 0.417 | Train Acc: 86.86%
	 Val. Loss: 0.951 |  Val. Acc: 78.95%
Epoch: 04 | Epoch Time: 1m 46s
	Train Loss: 0.398 | Train Acc: 87.42%
	 Val. Loss: 0.920 |  Val. Acc: 79.33%
Epoch: 05 | Epoch Time: 1m 47s
	Train Loss: 0.380 | Train Acc: 87.98%
	 Val. Loss: 0.893 |  Val. Acc: 79.74%
Epoch: 06 | Epoch Time: 1m 38s
	Train Loss: 0.364 | Train Acc: 88.37%
	 Val. Loss: 0.860 |  Val. Acc: 80.69%
Epoch: 07 | Epoch Time: 1m 35s
	Train Loss: 0.351 | Train Acc: 88.74%
	 Val. Loss: 0.839 |  Val. Acc: 81.02%
Epoch: 08 | Epoch Time: 1m 42s
	Train Loss: 0.339 | Train Acc: 89.09%
	 Val. Loss: 0.814 |  Val. Acc: 81.41%


In [105]:


model.load_state_dict(torch.load('blstm1_test.pt', map_location=torch.device('cpu')))

RuntimeError: Error(s) in loading state_dict for BiLSTMPOSTagger:
	size mismatch for embedding.weight: copying a param with shape torch.Size([7519, 100]) from checkpoint, the shape in current model is torch.Size([23626, 100]).